In [ ]:
import pandas as pd

# convert utm32 to lat long
import pyproj #

def utm_to_latlong(x, y):
    transformer = pyproj.Transformer.from_crs(32632, 4326, always_xy=True)
    lon, lat = transformer.transform(x, y)
    return pd.Series([lat, lon])

years = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014"]

for year in years:
    # read xlsx file ignore first 8 rows
    df = pd.read_excel('data/faste-trafiktaellinger-'+year+'.xlsx', skiprows=8)

    # join first two rows
    df.iloc[1] = df.iloc[0].apply(lambda x: str(x) + " " if (str(x)!="nan" and str(x)!="Talt") else "") + df.iloc[1].apply(lambda x: str(x) if str(x)!="nan" else "")

    # drop first row
    df = df.drop(df.index[0])

    # set column names to first row
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    # reset index
    df = df.reset_index(drop=True)

    # change type of all columns starting with "kl." to int
    for col in df.columns:
        if (col.startswith("X") or col.startswith("Y")):
            df[col] = df[col].apply(lambda x: int(x))
        if (col.startswith("kl.")):
            df[col] = df[col].apply(lambda x: int(x) if str(x)!="nan" else 0)

    df["Dato"] = pd.to_datetime(df["Dato"], format="%d.%m.%Y").dt.date

    # drop Spor column
    df = df.drop(columns=["Spor"])

    df[["lat", "lon"]] = df.apply(lambda row: utm_to_latlong(row["Xkoordinat (UTM32)"], row["Ykoordinat (UTM32)"]), axis=1)

    # combine lat lon column into list in one column
    df["latlon"]= df.apply(lambda row: (row["lat"], row["lon"]), axis=1)

    # drop xkoordinat and ykoordinat columns
    df = df.drop(columns=["Xkoordinat (UTM32)", "Ykoordinat (UTM32)"])

    # save to csv
    df.to_csv('data/traffic-'+year+'.csv', index=False, sep=',')
